In [3]:
pip install pyarrow


  Obtaining dependency information for pyarrow from https://files.pythonhosted.org/packages/cd/93/c2d3384aba712a0eb503f3940132189e81e97fb320844651783f45f15722/pyarrow-15.0.2-cp312-cp312-win_amd64.whl.metadata
   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.3 MB 660.6 kB/s eta 0:00:39
   ---------------------------------------- 0.1/25.3 MB 1.3 MB/s eta 0:00:20
   - -------------------------------------- 0.7/25.3 MB 5.6 MB/s eta 0:00:05
   ---- ----------------------------------- 2.6/25.3 MB 15.2 MB/s eta 0:00:02
   ------ --------------------------------- 4.0/25.3 MB 18.2 MB/s eta 0:00:02
   ------ --------------------------------- 4.3/25.3 MB 16.3 MB/s eta 0:00:02
   ------- -------------------------------- 4.9/25.3 MB 16.5 MB/s eta 0:00:02
   --------- ------------------------------ 5.9/25.3 MB 16.3 MB/s eta 0:00:02
   ---------- --------------


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
from plyer import notification
import requests
import pyodbc
from sqlalchemy import create_engine

# Função para exibir uma notificação na tela
def exibir_notificacao():
    # Título e mensagem da notificação
    titulo = "Alerta"
    mensagem = "Houve uma falha ao fazer a requisição à API. Verifique o status da conexão."
    
    # Exibir a notificação
    notification.notify(
        title=titulo,
        message=mensagem,
        timeout=10  # timeout define a duração da notificação
    )

# Função para fazer a requisição à API e extrair os dados de bancos
def extrair_dados_api(url):
    try:
        # Fazendo a requisição para a API
        response = requests.get(url)
        
        # Verificando se a requisição foi bem-sucedida (status code 200)
        if response.status_code == 200:
            # Convertendo a resposta para JSON
            data = response.json()

            # Extraindo dados para a tabela de bancos
            banks_table = [{"Nome": bank.get("name", ""), "Código": bank.get("code", ""), "ISPB": bank.get("ispb", "")} for bank in data]
            df_bancos = pd.DataFrame(banks_table)

            # Tratamento de valores ausentes para a tabela de bancos
            df_bancos.fillna({'Nome': 'Sem nome', 'Código': 'N/A', 'ISPB': 'Desconhecido'}, inplace=True)

            # Remover duplicatas, se houver, na tabela de bancos
            df_bancos.drop_duplicates(inplace=True)
            
            return df_bancos
        else:
            # Exibe uma notificação de alerta em caso de falha na requisição
            exibir_notificacao()
            return None
    except Exception as e:
        # Alerta em caso de erro na extração
        alerta(nivel=3, base="bancos", etapa="EXTRACAO", erro=e)
        return None

# Função para conectar ao banco de dados SQL Server
def conectar_bd(server, database, driver):
    try:
        # Crie a string de conexão com autenticação do Windows
        connection_string = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver={driver}'
        engine = create_engine(connection_string)
        return engine
    except Exception as e:
        alerta(nivel=3, base="bancos", etapa="BD", erro=e)
        return None

# Função para importar os dados extraídos para o SQL Server
def importar_para_sql(df_bancos, engine, table_name):
    try:
        # Importe o DataFrame para o SQL Server
        df_bancos.to_sql(table_name, con=engine, if_exists='replace', index=False)
    except Exception as e:
        alerta(nivel=3, base="bancos", etapa="IMPORTACAO", erro=e)

# Função principal para execução das etapas de extração e importação
def etapa_extracao_importacao():
    try:
        # URL da API
        url = "https://brasilapi.com.br/api/banks/v1"
        
        # Extrair dados da API
        df_bancos = extrair_dados_api(url)
        
        if df_bancos is not None:
            # Conexão com o banco de dados SQL Server
            server = 'DESKTOP-1MBU4JV\\SQLEXPRESS'
            database = 'PROJETO_PYTHON'
            driver = 'ODBC+Driver+17+for+SQL+Server'
            engine = conectar_bd(server, database, driver)
            
            if engine is not None:
                # Nome da tabela para importar os dados
                table_name = 'Bancos'
                
                # Importar os dados extraídos para o SQL Server
                importar_para_sql(df_bancos, engine, table_name)
                print("Importação concluída.")
            else:
                print("Falha ao conectar ao banco de dados.")
        else:
            print("Falha ao extrair dados da API.")
    except Exception as e:
        print("Ocorreu um erro durante a execução:", e)

# Executar as etapas de extração e importação
etapa_extracao_importacao()
# Conexão com o banco de dados SQL Server
server = 'DESKTOP-1MBU4JV\\SQLEXPRESS'
database = 'PROJETO_PYTHON'
driver = 'ODBC+Driver+17+for+SQL+Server'
engine = conectar_bd(server, database, driver)

# Executar as etapas de extração e importação
etapa_extracao_importacao()

# Verificar se a engine foi definida corretamente antes de carregar os dados da tabela
if engine is not None:
    # Exibir os primeiros registros da tabela importada
    df_bancos = pd.read_sql_table('Bancos', con=engine)
    print(df_bancos.head())
else:
    print("Falha ao conectar ao banco de dados.")

# Exibir os primeiros registros da tabela importada
df_bancos = pd.read_sql_table('Bancos', con=engine)
print(df_bancos.head())

Importação concluída.
Importação concluída.
                         Nome Código      ISPB
0          BCO DO BRASIL S.A.    1.0  00000000
1  BRB - BCO DE BRASILIA S.A.   70.0  00000208
2                       Selic    N/A  00038121
3                       Bacen    N/A  00038166
4       SANTINVEST S.A. - CFI  539.0  00122327
                         Nome Código      ISPB
0          BCO DO BRASIL S.A.    1.0  00000000
1  BRB - BCO DE BRASILIA S.A.   70.0  00000208
2                       Selic    N/A  00038121
3                       Bacen    N/A  00038166
4       SANTINVEST S.A. - CFI  539.0  00122327


In [8]:
# Query para dar erro

import pandas as pd
from plyer import notification
import requests
import pyodbc
from sqlalchemy import create_engine

# Função para exibir uma notificação na tela
def exibir_notificacao():
    # Título e mensagem da notificação
    titulo = "Alerta"
    mensagem = "Houve uma falha ao fazer a requisição à API. Verifique o status da conexão."
    
    # Exibir a notificação
    notification.notify(
        title=titulo,
        message=mensagem,
        timeout=10  # timeout define a duração da notificação
    )

# Função para fazer a requisição à API e extrair os dados de bancos
def extrair_dados_api(url):
    try:
        # Simulando um erro na requisição para a API
        raise Exception("Erro ao fazer a requisição à API")
        
        # Fazendo a requisição para a API
        response = requests.get(url)
        
        # Verificando se a requisição foi bem-sucedida (status code 200)
        if response.status_code == 200:
            # Convertendo a resposta para JSON
            data = response.json()

            # Extraindo dados para a tabela de bancos
            banks_table = [{"Nome": bank.get("name", ""), "Código": bank.get("code", ""), "ISPB": bank.get("ispb", "")} for bank in data]
            df_bancos = pd.DataFrame(banks_table)

            # Tratamento de valores ausentes para a tabela de bancos
            df_bancos.fillna({'Nome': 'Sem nome', 'Código': 'N/A', 'ISPB': 'Desconhecido'}, inplace=True)

            # Remover duplicatas, se houver, na tabela de bancos
            df_bancos.drop_duplicates(inplace=True)
            
            return df_bancos
        else:
            # Exibe uma notificação de alerta em caso de falha na requisição
            exibir_notificacao()
            return None
    except Exception as e:
        # Alerta em caso de erro na extração
        print("Erro durante a extração dos dados da API:", e)
        exibir_notificacao()  # Exibir notificação de alerta
        return None

# Função para conectar ao banco de dados SQL Server
def conectar_bd(server, database, driver):
    try:
        # Crie a string de conexão com autenticação do Windows
        connection_string = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver={driver}'
        engine = create_engine(connection_string)
        return engine
    except Exception as e:
        print("Erro ao conectar ao banco de dados:", e)
        alerta(nivel=3, base="bancos", etapa="BD", erro=e)
        return None

# Função para importar os dados extraídos para o SQL Server
def importar_para_sql(df_bancos, engine, table_name):
    try:
        # Importe o DataFrame para o SQL Server
        df_bancos.to_sql(table_name, con=engine, if_exists='replace', index=False)
    except Exception as e:
        print("Erro durante a importação dos dados para o SQL Server:", e)
        alerta(nivel=3, base="bancos", etapa="IMPORTACAO", erro=e)

# Função principal para execução das etapas de extração e importação
def etapa_extracao_importacao():
    try:
        # URL da API
        url = "https://brasilapi.com.br/api/banks/v1"
        
        # Extrair dados da API
        df_bancos = extrair_dados_api(url)
        
        if df_bancos is not None:
            # Conexão com o banco de dados SQL Server
            server = 'DESKTOP-1MBU4JV\\SQLEXPRESS'
            database = 'PROJETO_PYTHON'
            driver = 'ODBC+Driver+17+for+SQL+Server'
            engine = conectar_bd(server, database, driver)
            
            if engine is not None:
                # Nome da tabela para importar os dados
                table_name = 'Bancos'
                
                # Importar os dados extraídos para o SQL Server
                importar_para_sql(df_bancos, engine, table_name)
                print("Importação concluída.")
            else:
                print("Falha ao conectar ao banco de dados.")
        else:
            print("Falha ao extrair dados da API.")
    except Exception as e:
        print("Ocorreu um erro durante a execução:", e)

# Executar as etapas de extração e importação
etapa_extracao_importacao()

Erro durante a extração dos dados da API: Erro ao fazer a requisição à API
Falha ao extrair dados da API.
